<a href="https://colab.research.google.com/github/kemusiro/ProgrammingGakuen-PythonClub/blob/main/202301/encrypt_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libmpc-dev
!pip install gmpy2

from PIL import Image, ImageDraw
import IPython
from math import sqrt, sin, cos, pi
import struct
from gmpy2 import mpz, powmod, gcdext

# 素数の組p, qから秘密鍵と公開鍵を生成する。
def genkey(p, q, e=65537):
    n = p * q
    m = (p - 1) * (q - 1)
    _, d, _ = gcdext(e, m)
    # 秘密鍵のdが正になるように変換する。
    while d < 0:
        d += m
    return (n, e), (n, int(d))

# 数値vを公開鍵を使って暗号化する。
def encrypt(v, pubkey):
    return list(map(lambda v: int(powmod(v, pubkey[1], pubkey[0])), v))

# 数値vを秘密鍵を使って復号する。
def decrypt(v, pubkey):
    return list(map(lambda v: int(powmod(v, prvkey[1], prvkey[0])), v))

pubkey, prvkey = genkey(3, 11, e=3)
a = [13]
b = encrypt(a, pubkey)
adash = decrypt(b, prvkey)
print(f'公開鍵={pubkey}, 秘密鍵={prvkey}, 平文={a}, 暗号文={b}, 復号={adash}')

# 進数を変換する。
def convert_base(src_digits, src_base, dest_base):
    x = mpz(0)
    for v in src_digits:
        x = x * src_base + v
    dest_digits = []
    while x > 0:
        x, r = divmod(x, dest_base)
        dest_digits.append(r)
    dest_digits.reverse()
    return dest_digits

##### 暗号化する画像を生成する。#####
width, height = 256, 256
img_original = Image.new("RGB", (width, height), (0, 0, 0))
d = ImageDraw.Draw(img_original)

r1 = sqrt(5 + 2 * sqrt(5)) / 4
r2 = r1 * (3 - sqrt(5)) / 2
scale = width / ((1 + sqrt(5)) / 2)

pos = []
for i in range(5):
    theta = 2 * pi * i / 5
    pos.append((scale * r1 * cos(theta) + width / 2,
                scale * r1 * sin(theta) + height / 2))
    pos.append((scale * r2 * cos(theta + pi / 5) + width / 2,
                scale * r2 * sin(theta + pi / 5) + height / 2))
d.polygon(pos, fill=(255, 0, 0))

print('オリジナルの画像')
IPython.display.display(img_original)

##### 画像を暗号化する。#####
pubkey, prvkey = genkey(3571, 3559)
print(f'公開鍵 = {pubkey}, 秘密鍵 = {prvkey}')

input_base4G = [v[0] for v in struct.iter_unpack('>I', img_original.tobytes())]
# 先頭が必ず非ゼロになるようにするためも兼ねて画像のサイズを先頭に挿入する。
input_base4G[0:0] = img_original.size
# 公開鍵中のNを用いて4G進数(32bit値の列)をN進数に変換する。
# これにより暗号化でmod Nを求めたときに重複無く変換できる。
input_baseN = convert_base(input_base4G, 256**4, pubkey[0])
encrypted_baseN = encrypt(input_baseN, pubkey)
encrypted_base4G = convert_base(encrypted_baseN, pubkey[0], 256**4)

raw_data = b''.join([struct.pack('>I', v) for v in encrypted_base4G])
line_size = img_original.size[0] * 3  # RGBの3色分
if len(raw_data) % line_size != 0:
    raw_data += b'\x00' * (line_size - len(raw_data) % line_size)
img_encrypted = Image.frombytes(
    "RGB",
    (line_size // 3, len(raw_data) // line_size),
    raw_data)

# 表示
print('暗号化された画像')
IPython.display.display(img_encrypted)

##### 画像を復号する。#####
decrypted_baseN = decrypt(encrypted_baseN, prvkey)
decrypted_base4G = convert_base(decrypted_baseN, prvkey[0], 256**4)
w, h = decrypted_base4G[0], decrypted_base4G[1]
output = b''.join([struct.pack('>I', v) for v in decrypted_base4G[2:]])

img_decrypted = Image.frombytes("RGB", (w, h), bytes(output))
print('復号された画像')
IPython.display.display(img_decrypted)